In [5]:
from groq import Groq
from dotenv import load_dotenv
import cohere
import json
import os
load_dotenv()

True

In [6]:
client = Groq( api_key=os.getenv("GROQ_API_KEY"))


In [7]:
co = cohere.ClientV2(api_key="txPrIh5q28qkvynGY5b4Uimtoff5eLTINAvwrV2v")

# Test Cohere embeddings


In [10]:

def get_chatbot_response(client,messages,model_name="llama-3.1-8b-instant",temperature=0):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "content": message["content"]})

    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=temperature,
        top_p=0.8,
        max_tokens=2000,
    ).choices[0].message.content
    
    return response

# Prompt Engineering

# Structure Output


In [21]:
system_prompt = """
    You are a helpful assistant that answers questions about capitals of countries.

    Your Output should be in structured JSON format.You should not include anything other than json object in your output.
  [
{
    country: the country that you will get the capital of 
    capital: the capital of the country stated
}
]
    """


messages = [
    {
        "role": "system",
        "content": system_prompt,
    }
]


# input Structure

In [23]:
user_prompt = """
Get me the capitals of the following countries:
```
1. Italy
2. France
3. Germany
``
"""
messages = [{'role':'system','content':system_prompt}]
messages.append({'role':'user','content':user_prompt})
response = get_chatbot_response(client,messages)
print(response)

[
  {
    "country": "Italy",
    "capital": "Rome"
  },
  {
    "country": "France",
    "capital": "Paris"
  },
  {
    "country": "Germany",
    "capital": "Berlin"
  }
]


# Give The Model Chain OF Thoughts

In [26]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    steps: This is where you solve the equation bit by bit following the BEDMAS order of operations. You need to show your work and calculate each step leading to final result. Feel free to write here in free text. 
    result: The final number resulted from calculating the equation above
}
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,messages)

In [27]:
print(response)

{
    "steps": "First, we need to follow the BEDMAS order of operations. 
    1. Divide 259 by 2: 259/2 = 129.5
    2. Multiply 129.5 by 8654: 129.5 * 8654 = 1121011
    3. Multiply 91072 by 33: 91072 * 33 = 3005016
    4. Add 1121011 and 3005016: 1121011 + 3005016 = 4126027
    5. Subtract 12971 from 4126027: 4126027 - 12971 = 4113056",
    "result": 4113056
}


# RAG


In [8]:
def get_embedding(text_input):
    response = co.embed(
        texts=text_input,
        model='embed-english-v3.0',
        input_type='search_document',
        embedding_types=["float"]
    )
    
    return response.embeddings.float_

In [9]:
get_embedding(["hello","goodbye"])

[[-0.0027694702,
  0.017807007,
  -0.04046631,
  -0.06402588,
  0.011924744,
  -0.016296387,
  -0.027801514,
  0.002357483,
  0.0009570122,
  0.051818848,
  -0.024734497,
  -0.02708435,
  0.030334473,
  -0.032440186,
  0.018112183,
  -7.4505806e-05,
  0.036315918,
  0.034423828,
  0.005115509,
  -0.012664795,
  -0.049102783,
  0.023101807,
  -0.02078247,
  0.021194458,
  0.036315918,
  0.026977539,
  -0.0039863586,
  -0.0019874573,
  -0.025634766,
  -0.035614014,
  0.008979797,
  0.007663727,
  0.028778076,
  0.02482605,
  -0.0016870499,
  0.06726074,
  -0.019515991,
  -0.0048294067,
  0.007896423,
  0.0017938614,
  0.026550293,
  -0.025939941,
  -0.07409668,
  -0.028656006,
  -0.07525635,
  -0.025512695,
  -0.0037765503,
  0.0031414032,
  0.029586792,
  -0.030029297,
  0.00084114075,
  0.0037002563,
  -0.0077705383,
  0.059692383,
  -0.016540527,
  0.0088272095,
  -0.060821533,
  -0.018218994,
  -0.0014333725,
  0.035125732,
  0.029891968,
  -0.008842468,
  0.03942871,
  0.04458618,
 